In [2]:
%matplotlib inline
import pandas as pd
import numpy as np

import os

In [3]:
votes = pd.DataFrame.from_csv('VotesRaw.csv')
votes.index = range(votes.shape[0])
print votes.shape

(27041, 17)


In [4]:
# Note: there are some spurious entries Ans with only one entry in the whole dataset.
feats = ['Ans_count',"QuestionId","AnsRank","Votes","ReScore","Norm_Pos","Norm_Pos_2","Norm_DRank","EPbias"]
## preserves index
get_row_at = lambda df,at: pd.DataFrame(dict(zip(df.iloc[at].index,[[v] for v in df.iloc[at].values])),index=[df.iloc[at].name])
get_arrival = lambda df: get_row_at(df[feats],1) if df.shape[0]>1 else pd.DataFrame()
first_arrivals = votes.sort_values(by='Age').groupby(['AnsId']).apply(get_arrival).reset_index(level=[0]
                                                                                                   ,drop=False)
fst_arr_idx = first_arrivals.index

In [5]:
fst_arr_idx.size

4759

In [6]:
np.unique(votes.QuestionId.values).size

2516

## For regression, take Qid and time at which ans arrives

In [7]:
# can take into account more than one arrival if happens the same day

has_first_arrival = lambda df: np.intersect1d(df.index.values,fst_arr_idx.values).size==1
qid_age_fstarr = votes.groupby(['QuestionId','Age']).apply(
    lambda df: get_row_at(df[['QuestionId','Age']],0)
        if has_first_arrival(df) else pd.DataFrame()).reset_index(level=[0,1],drop=True)
votes_from_fstarr = pd.merge(qid_age_fstarr,votes,how='inner',on=['QuestionId'],suffixes=['_from','_curr'])
votes_from_fstarr.shape

(54939, 18)

In [8]:
#qid_age_fstarr['Age'].value_counts()

In [9]:
feats_reg = ['QuestionId','Age_curr',
             'Ans_count','AnsId','Votes','AnsRank','ReScore', 'Norm_Pos','Norm_Pos_2','Norm_DRank','EPbias']
votes_from_fstarr = votes_from_fstarr[votes_from_fstarr['Age_curr']>=votes_from_fstarr['Age_from']]
votes_from_fstarr = votes_from_fstarr[feats_reg].drop_duplicates()

In [10]:
votes_from_fstarr.head()

,QuestionId,Age_curr,Ans_count,AnsId,Votes,AnsRank,ReScore,Norm_Pos,Norm_Pos_2,Norm_DRank,EPbias
7,1,2,4,56,1,1,0.993820,0.25,0.0625,0.00,0.328707
8,1,2,4,210,1,2,0.981460,0.50,0.2500,0.00,0.189274
9,1,2,4,4,0,3,0.490730,0.75,0.5625,-0.25,0.218297
10,1,2,4,5,0,4,0.981460,1.00,1.0000,0.25,0.263722
11,1,5,4,56,1,1,0.994703,0.25,0.0625,0.00,0.328707


In [15]:
for nb_choices_nnet in range(2,7):
    nb_choices_nnet = 6
    for_nnet = pd.DataFrame()
    for k,g in votes_from_fstarr[votes_from_fstarr['Ans_count']==nb_choices_nnet].groupby(['Votes']):
        if k>1:
            for_nnet = for_nnet.append([g]*(int(k-1)),ignore_index=True)
        else:
            for_nnet = for_nnet.append(g,ignore_index=True)
    for_nnet.loc[for_nnet.Votes>1,'Votes']=1
    for_nnet = for_nnet[for_nnet.Votes>0]
    for_nnet.to_csv('Qtnnet%dFirstArr.csv'%nb_choices_nnet)  

## For simulation take exactly the question state when ans arrives

In [29]:
# can take into account more than one arrival if happens the same day
feats = ['Ans_count',"AnsRank","Votes","ReScore","Norm_Pos","Norm_Pos_2","Norm_DRank"]
has_first_arrival = lambda df: np.intersect1d(df.index.values,fst_arr_idx.values).size==1
votes_on_fstarr = votes.groupby(['QuestionId','Age_vot']).apply(
    lambda df: df[feats] if has_first_arrival(df) else pd.DataFrame()).reset_index(level=[0,1],drop=False)

In [112]:
nb_choices_nnet = 6
for_nnet = pd.DataFrame()
for k,g in votes_on_fstarr[votes_on_fstarr['Ans_count']==nb_choices_nnet].groupby(['Votes']):
    if k>1:
        for_nnet = for_nnet.append([g]*(int(k-1)),ignore_index=True)
    else:
        for_nnet = for_nnet.append(g,ignore_index=True)
for_nnet.loc[for_nnet.Votes>1,'Votes']=1
for_nnet = for_nnet[for_nnet.Votes>0]
for_nnet.to_csv('VotesOnArr%d.csv'%nb_choices_nnet)

In [113]:
#non_fst_idx = np.setdiff1d(votes.index.values,fst_arr_idx)
#train = votes.loc[votes.index.isin(non_fst_idx),:]